In [2]:
!pip freeze | grep scikit-learn

scikit-learn @ file:///tmp/build/80754af9/scikit-learn_1642617106979/work
scikit-learn-intelex==2021.20220215.212715


In [14]:
!pip list | grep scikit-learn

scikit-learn                  1.0.2
scikit-learn-intelex          2021.20220215.212715


In [17]:
!pip list | grep pyarrow

pyarrow                       16.1.0


: 

In [3]:
!python -V

Python 3.9.12


In [4]:
import pickle
import pandas as pd

In [5]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

/home/codespace/anaconda3/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/codespace/anaconda3/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [6]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [7]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')

In [8]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

What's the standard deviation of the predicted duration for this dataset?

In [9]:
y_pred.std()

6.247488852238703

What's the size of the output file?

In [18]:
#df['year']

In [10]:
df['ride_id'] = f'2023/03_' + df.index.astype('str')

In [11]:
output_file = '2023_03_yellow_taxi_predictions.parquet'

df.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [12]:
df.shape

(3316216, 21)

In [13]:
import pyarrow.parquet as pq

parquet_file = pq.ParquetFile(output_file)
num_rows = parquet_file.metadata.num_rows
print(f'Number of rows: {num_rows}')

Number of rows: 3316216
